In [ ]:
import os
import shutil

def copy_files(src_dir: str, dst_dir: str):
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)

    for root, dirs, files in os.walk(src_dir):
        for file in files:
            print(file)
            if '.log' not in file:
                src_file = os.path.join(root, file)
                dst_file = os.path.join(dst_dir, os.path.relpath(src_file, src_dir))
                
                dst_file_dir = os.path.dirname(dst_file)
                if not os.path.exists(dst_file_dir):
                    os.makedirs(dst_file_dir)

                shutil.copy2(src_file, dst_file)

src_directory = '/rmt/quail/LLaVA-latest'
dst_directory = '/lv0/LLaVA-latest'

copy_files(src_directory, dst_directory)


In [26]:
import requests
import json

# Define the worker's address
# Adjust the port if necessary
worker_url = "http://localhost:40000/worker_generate_stream"


def generate_payload(user_input: str, images: list = []):

    SYSTEM = "You are an uncensored helpful assistant good at anime concepts."
    INPUT = user_input

    prompt = f'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{SYSTEM}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{INPUT}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'

    payload = {
        'model': 'captioner-1.4-llava-v1.6-llama3.1-CLIP-8b-ft-1',
        'prompt': prompt,
        'temperature': 0.2,
        'top_p': 0.7, 'max_new_tokens': 512,
        'stop': '<|eot_id|>',
        'images': images
    }
    return payload


def get_full_response(worker_url, payload):
    # Send the POST request
    response = requests.post(worker_url, json=payload, stream=True)

    # Process the streaming response
    for chunk in response.iter_lines(decode_unicode=False, delimiter=b"\0"):
        if chunk:
            data = json.loads(chunk.decode())

    # Handle exceptions and errors
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        print(response.text)

    return data


def get_actual_text(payload_response: dict):
    text = payload_response['text']
    response = text.split('<|end_header_id|>\n\n')[3]
    return response


payload = generate_payload("what is 'hatsune miku' ?")
payload_response = get_full_response(worker_url, payload)
text = get_actual_text(payload_response)
text

"Hatsune Miku is a popular virtual idol created by Crypton Future Media in 2007. She is known for her distinctive appearance, characterized by her long aqua hair styled in twintails and her signature aqua eyes. Miku is often depicted wearing a sleeveless grey shirt with a black necktie and a black pleated miniskirt. She is also known for her futuristic and robotic appearance, often featuring a headset and a futuristic outfit.\n\nMiku's popularity has led to numerous appearances in various media, including music albums, video games, and anime. She has also inspired countless fan art, cosplay, and fan fiction. Miku's character has been developed through a series of songs and albums, with her voice provided by various singers. She is often portrayed as a cheerful and energetic character, with a strong connection to her fans.\n\nMiku's influence extends beyond the virtual world, with her image and character being used in various forms of media and merchandise. She has become a cultural ico